In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import time
import datetime

In [2]:
amzurl = "https://www.amazon.com/s?k=gaming+table&rh=n%3A1055398%2Cp_72%3A1248915011&dc&ds=v1%3AGX6Xkgs25SPqHK358dqkMmTpuDu%2BGJWhNr9B8WunBPA&qid=1706455511&rnid=1248913011&ref=sr_nr_p_72_1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/106.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
amzpage = requests.get(amzurl,headers=headers)
amzcode = BeautifulSoup(amzpage.content, "html.parser")

In [3]:
#this is to see exactly what python sees, because depending on preset modifications to the website such as preferred location or
#currency, what is seen by clicking the link is different from what python sees by scraping
with open('amzcode.html', 'w', encoding='UTF8') as f:
    f.write(str(amzcode))

In [4]:
data1 = amzcode.find_all('span', class_='a-size-base-plus a-color-base a-text-normal')
amztitles = []
for x in range(len(data1)):
    amztitles.append(amzcode.find_all('span', class_='a-size-base-plus a-color-base a-text-normal')[x].get_text())
#amztitles

In [5]:
data2 = amzcode.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
amzlinks = []
for x in range(len(data2)):
    amzlinks.append('https://www.amazon.com'+ amzcode.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')[x].get('href'))
#amzlinks

In [16]:
#for some reason there was an inconsistency with the prices after the first 2 entries so it will be scraped from each
#product's link directly
amzprices = []
for x in range(len(data2)):
    eachamzpage = requests.get(amzlinks[x],headers=headers)
    eachamzcode = BeautifulSoup(eachamzpage.content, "html.parser")
    data3 = eachamzcode.find_all('span', class_='a-offscreen')
    amzprices.append(eachamzcode.find('span', class_='a-offscreen').get_text())
#some ads on the website come without a price so the script returns something completely random
for x in range(len(amzprices)):
    if amzprices[x][0] != '$':
        amzprices[x] = 'no price displayed'
#amzprices

In [7]:
data3 = amzcode.find_all('span', class_='a-size-base s-underline-text')
amzvotes = []
for x in range(len(data3)):
    amzvotes.append(amzcode.find_all('span', class_='a-size-base s-underline-text')[x].get_text())
#amzvotes

In [17]:
import csv
header = ['Title', 'Price', 'Link', 'Review Count']
with open("webscraping project1-Amazon.csv", 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(zip(amztitles, amzprices, amzlinks, amzvotes))